In [ ]:
import numpy as np
import torch
import json

In [ ]:
import train_evaluate

In [ ]:
# fix random seeds
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# define path to dataset
data_path = '../data/Full-Data-80-20-split-6000.json'

# get device
device = train_evaluate.get_device()

In [ ]:
# define hyperparameters for training
HIDDEN_LAYERS = 1
HIDDEN_FEATURES = 512

BATCH_SIZE = 512
NUM_EPOCHS = 100
LR = 0.0008

# train model
model, summary = train_evaluate.train_model(
    data_path=data_path,
    lr=LR,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    hidden_layers=HIDDEN_LAYERS,
    hidden_features=HIDDEN_FEATURES,
    final_plot=True
)

# print summary
train_evaluate.print_summary(summary)

In [ ]:
# save model
model_path = '../trained_models/example_model'
train_evaluate.save_model_summary(model, summary, path=model_path)

Second part: Load a trained model and use it to compute assignments.

1. load model
2. classify
```
trained_model, _ = train_evaluate.load_model_for_inference(PATH)
assignment = train_evaluate.classify(trained_model, x, 'one_hot') 
```
classify() works with lists, numpy arrays and torch tensors. Just make sure, the dimensions are correct. The most general input shape is [batch_size, num_kitchens + 1, num_items]. Specify the output format by passing for example 'one_hot' or 'label' to the classify method, see examples below. 

In [ ]:
# load trained model to perform inference
trained_model, loaded_summary = train_evaluate.load_model_for_inference(model_path)

# print training summary
train_evaluate.print_summary(loaded_summary)

# print model architecture
print('Model architecture:')
print(trained_model)

In [ ]:
# load full data from json file and convert to numpy arrays
f = open(data_path)
full_data = json.load(f)
food_data = np.array(full_data['food_data_train'])  # [num_examples, num_kitchens + 1, num_items]
true_assignments = np.array(full_data['kitchens_data_train'])  # [num_examples, num_kitchens]

In [ ]:
# classify full dataset and compute accuracy
pred = train_evaluate.classify(trained_model, food_data, 'label')
accuracy = np.sum(np.where(pred - np.argmax(true_assignments, axis=1) == 0, 1, 0)) / len(food_data) * 100
print(f'Accuracy over training dataset: {accuracy :.2f}%')

In [ ]:
# generate a single test example with dimensions [num_kitchens + 1, num_items]
num_kitchens = loaded_summary['num_kitchens']
num_items = loaded_summary['num_items']

# random kitchen inventory
means = np.mean(food_data, axis=0)[:num_kitchens, :]
stds = np.std(food_data, axis=0)[:num_kitchens, :]
random_inventory = stds * np.random.randn(num_kitchens, num_items) + means

# one hot item to distribute
random_item_to_distribute = np.zeros(num_items)
random_item_to_distribute[np.random.choice(num_items)] = 100

# combine inventory and item to distribute
test_input = np.vstack((random_inventory, random_item_to_distribute))
print(f'Shape of test input: {test_input.shape}')

In [ ]:
# Compute assignments with different output encodings
assignment_default = train_evaluate.classify(trained_model, test_input)
print(f'Assignments with probabilities: {assignment_default}')

assignment_one_hot = train_evaluate.classify(trained_model, test_input, 'one_hot')
print(f'Assignments one hot encoded: {assignment_one_hot}')

assignment_label = train_evaluate.classify(trained_model, test_input, 'label')
print(f'Assignments with kitchen labels: {assignment_label}')